In [1]:
import os
os.environ["HOME"] = "/mnt/nas/shuvranshu"

os.environ["HF_HOME"] = "/mnt/nas/shuvranshu/huggingface_cache"
os.environ["TRANSFORMERS_CACHE"] = "/mnt/nas/shuvranshu/huggingface_cache"
os.environ["HF_DATASETS_CACHE"] = "/mnt/nas/shuvranshu/huggingface_cache"
os.environ["XDG_CACHE_HOME"] = "/mnt/nas/shuvranshu/huggingface_cache"
os.environ["HF_DATASETS_CACHE"] = "/mnt/nas/shuvranshu/huggingface_cache"

os.makedirs("/mnt/nas/shuvranshu/huggingface_cache", exist_ok=True)



In [2]:
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
#hf token 
load_dotenv()  
hf_token = os.getenv("HF_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

/mnt/nas/shuvranshu/.conda/envs/ragenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/nas/shuvranshu/.conda/envs/ragenv/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
llm = HuggingFacePipeline.from_model_id(
    # model_id="/mnt/nas/shuvranshu/huggingface_cache/models--meta-llama--Llama-3.1-8B/snapshots/d04e592bb4f6aa9cfee91e2e20afa771667e1d4b", 
    model_id="meta-llama/Llama-3.1-8B",
    # model_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    model_kwargs={"temperature": 0.1},
    device=2
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,     # characters per chunk
    chunk_overlap=50,   # overlap to preserve context
    separators=["\n\n", "\n", " ", ""]
)



embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device": "cuda:2"})





The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.25it/s]
Device set to use cuda:2
/tmp/ipykernel_286326/1735494884.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device": "cuda:2"})


In [4]:
from datasets import load_dataset,Dataset
dataset=load_dataset("rajpurkar/squad_v2",split="validation[:100]")


In [5]:
print(dataset[0])
print(dataset[99])


{'id': '56ddde6b9a695914005b9628', 'title': 'Normans', 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.', 'question': 'In what country is Normandy located?', 'answers': {'text': ['France', 'France', 'France', 'France'], 'answer_start': [159, 159, 159, 159]}}
{'id': '5ad3f4b1604f3c001a3ff951', 

In [6]:
#KG implementation
from dataclasses import dataclass
from typing import List, Dict

@dataclass
class Triple:
    subj: str
    rel: str
    obj: str

class SimpleKG:
    def __init__(self):
        self.triples: List[Triple] = []

    def add_triple(self, subj: str, rel: str, obj: str):
        self.triples.append(Triple(subj, rel, obj))

    def find_triples(self, entity: str) -> List[Triple]:
        # return all triples where entity is subject or object
        return [t for t in self.triples if t.subj == entity or t.obj == entity]


KG = SimpleKG()
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_triples_spacy(text):
    doc = nlp(text)
    triples = []
    for token in doc:
        if token.dep_ in ("ROOT", "relcl"):  # verbs or relations
            subj = [w.text for w in token.lefts if w.dep_ in ("nsubj", "nsubjpass")]
            obj = [w.text for w in token.rights if w.dep_ in ("dobj", "pobj", "attr")]
            if subj and obj:
                triples.append((" ".join(subj), token.lemma_, " ".join(obj)))
    return triples




#link entities in query to KG entities
import spacy
nlp = spacy.load("en_core_web_sm")
def extract_entity_mentions(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents] or [chunk.text for chunk in doc.noun_chunks]

def link_entities(query, kg_entities):
    # simple substring + optional embedding similarity
    mentions = extract_entity_mentions(query)  
    entity_map = {}
    for m in mentions:
        entity_map[m] = [e for e in kg_entities if m.lower() in e.lower()]
    return entity_map

def retrieve_kg_context(query, KG: SimpleKG):
    kg_entities = list(set([t.subj for t in KG.triples] + [t.obj for t in KG.triples]))
    entity_map = link_entities(query, kg_entities)
    triples_text = []
    for ents in entity_map.values():
        for ent in ents:
            for t in KG.find_triples(ent):
                triples_text.append(f"{t.subj} {t.rel} {t.obj}")
    return "\n".join(triples_text)


#combine kg retrieval and context retrieval
def get_combined_context(query, retriever, KG):
    # 1. Retrieve text from Chroma DB
    text_docs = retriever.get_relevant_documents(query)
    text_context = "\n\n".join([d.page_content for d in text_docs])

    # 2. Retrieve KG triples
    kg_context = retrieve_kg_context(query, KG)

    # 3. Combine for final LLM input
    combined_context = f"KG Facts:\n{kg_context}\n\nTextual Context:\n{text_context}"
    return combined_context

In [6]:
type(dataset[0]["context"])

str

In [7]:
#getting questions,ground_truths,adding context to vectorstore and making KG
questions=[]
ground_truths=[]
doc=""
q=0
for row in dataset:
    questions.append(row["question"])
    doc=doc+row["context"]+"\n"
    if(len(row["answers"]["text"])):
        ground_truths.append(row["answers"]["text"][0])
    else:
        ground_truths.append("")
    triples = extract_triples_spacy(row["context"])
    print(f"triple {q}:{triples}")
    # print(row["context"])
    for (subj, rel, obj) in triples:
        KG.add_triple(subj.strip(), rel.strip(), obj.strip())
    print(f"question:{q} completed")
    q+=1
#chunk the doc and add to vectorstore
chunks = text_splitter.split_text(doc)
vectorstore = Chroma.from_texts(
    texts=chunks,
    embedding=embeddings,
    collection_name="my_rag_knowledge"
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})



triple 0:[('French', 'be', 'people'), ('who', 'in', 'centuries')]
question:0 completed
triple 1:[('French', 'be', 'people'), ('who', 'in', 'centuries')]
question:1 completed
triple 2:[('French', 'be', 'people'), ('who', 'in', 'centuries')]
question:2 completed
triple 3:[('French', 'be', 'people'), ('who', 'in', 'centuries')]
question:3 completed
triple 4:[('French', 'be', 'people'), ('who', 'in', 'centuries')]
question:4 completed
triple 5:[('French', 'be', 'people'), ('who', 'in', 'centuries')]
question:5 completed
triple 6:[('French', 'be', 'people'), ('who', 'in', 'centuries')]
question:6 completed
triple 7:[('French', 'be', 'people'), ('who', 'in', 'centuries')]
question:7 completed
triple 8:[('French', 'be', 'people'), ('who', 'in', 'centuries')]
question:8 completed
triple 9:[('dynasty', 'have', 'impact'), ('They', 'adopt', 'language'), ('Duchy', 'be', 'fief'), ('prince', 'found', 'Principality')]
question:9 completed
triple 10:[('dynasty', 'have', 'impact'), ('They', 'adopt', 'l

In [8]:

rag_answers=[]
retrieved_contexts=[]
q=0



from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a factual assistant. Use the following context to answer the question.
Do NOT add information that is not supported by the context.

Context:
{context}

Question: {question}
Answer:
"""
)

from langchain.chains import LLMChain

llm_chain = LLMChain(
    llm=llm,       # your LLM object
    prompt=prompt
)

for question in questions:
    context = get_combined_context(question,retriever, KG)
    retrieved_contexts.append(context)
    response = llm_chain.run({
        "context": context,
        "question": question
    })

    rag_answers.append(response)
    answer=response.split('Answer:')[-1].strip()
    print(f"qa {q}:{answer}")
    print("...........................")
    q+=1



/tmp/ipykernel_286326/2854699044.py:25: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
/tmp/ipykernel_286326/1953730636.py:71: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  text_docs = retriever.get_relevant_documents(query)
/tmp/ipykernel_286326/2854699044.py:33: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run({


qa 0:France

Explanation:
The Duchy of Normandy, which began in 911 as a fiefdom, was established by the treaty of Saint-Clair-sur-Epte between King Charles III of West Francia and the famed Viking ruler Rollo, and was
France

Explanation:
The Duchy of Normandy, which began in 911 as a fiefdom, was established by the treaty of Saint-Clair-sur-Epte between King Charles III of West Francia and the famed Viking ruler Rollo, and was
France

Explanation:
The Duchy of Normandy, which began in 911 as a fiefdom, was established by the treaty of Saint-Clair-sur-Epte between King Charles III of West Francia and the famed Viking ruler Rollo, and was
France

Explanation:
The Duchy of Normandy, which began in 911 as a fiefdom, was established by the treaty of Saint-Clair-sur-Epte between King Charles III of West Francia and the famed Viking ruler Rollo, and was
France

Explanation:
The Duchy of Normandy, which began in 911 as a fiefdom, was established by the treaty of Saint-Clair-sur-Epte between 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


qa 9:William II of Normandy
...........................
qa 10:Richard I of Normandy
...........................
qa 11:
...........................
qa 12:The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman,

Explanation:
This is a factual assistant. The context is the text. The question is what type of major impact did the Norman dynasty have on medieval Europe. The answer is a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Roma

In [ ]:
# import pandas as pd

# data = pd.DataFrame({
#     "user_input": questions,
#     "retrieved_contexts": retrieved_contexts,
#     "response": rag_answers,
#     "reference": ground_truths
    
# })

In [9]:
#ragas evaluation
dataset = []

for q, ctx, ans, ref in zip(questions, retrieved_contexts, rag_answers, ground_truths):
    dataset.append({
        "user_input": q,
        "retrieved_contexts": ctx if isinstance(ctx, list) else [ctx],
        "response": ans,
        "reference": ref
    })

In [10]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [11]:
from langchain_community.chat_models import ChatOllama
from ragas import evaluate
from langchain_community.embeddings import OllamaEmbeddings
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness


In [12]:

langchain_llm= ChatOllama(model="mistral")

langchain_embeddings=OllamaEmbeddings(model="nomic-embed-text")

/tmp/ipykernel_286326/54446920.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  langchain_llm= ChatOllama(model="mistral")
/tmp/ipykernel_286326/54446920.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  langchain_embeddings=OllamaEmbeddings(model="nomic-embed-text")


In [13]:
result = evaluate(dataset=evaluation_dataset,
                  metrics=[ LLMContextRecall(), Faithfulness(),FactualCorrectness()],
                  llm=langchain_llm,embeddings=langchain_embeddings)
print(result)

Evaluating:  12%|█▏        | 36/300 [02:39<18:23,  4.18s/it] Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt n_l_i_statement_prompt failed to parse output: The output parser failed to parse the output including retries.
Exception raised in Job[8]: RagasOutputParserException(The output parser failed to parse the output including retries.)
Evaluating:  16%|█▌        | 47/300 [03:23<17:19,  4.11s/it]Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to parse the output including retries.
Prompt fix_output_format failed to parse output: The output parser failed to

{'context_recall': 0.4201, 'faithfulness': 0.8413, 'factual_correctness(mode=f1)': 0.6252}
